# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [1]:
import json
import os
from typing import Sequence, List

import openai
from dotenv import load_dotenv
from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

load_dotenv("../../../.env")
assert (key := os.getenv("OPENAI_API_KEY")); openai.api_key = key
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [4]:
from llama_index.agent import OpenAIAgent
from llama_index.agent.experimental_openai_agent import (
    ExperimentalOpenAIAgent, ChatStreamHandler
)
from llama_index.llms import OpenAI

In [5]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
csh = ChatStreamHandler(llm)
exp_agent = ExperimentalOpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    stream_handler=csh,
    verbose=True
)

In [6]:
print(agent.chat("Tell me a random name"))

Sure! Here's a random name: "Elijah"


In [7]:
print(agent.chat("Now tell me that name backwards."))

The name "Elijah" spelled backwards is "hjaile".


In [8]:
response = exp_agent.chat("What is (121 * 3) + 42?")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405


### Chat

In [9]:
response = exp_agent.chat("What is (121 * 3) + 42?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405
(121 * 3) + 42 is equal to 405.


In [10]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [11]:
response = await exp_agent.achat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 multiplied by 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [12]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
The answer to 121 multiplied by 2 is 242.

Once upon a time, in a cozy little village, there was a group of 242 mice who lived harmoniously in a charming mouse hole. These mice were not your ordinary mice; they were known for their intelligence and resourcefulness.

The leader of the mouse group was a wise and courageous mouse named Milo. Milo was admired by all the mice for his ability to solve problems and lead the group through any challenge they faced. Under his guidance, the mice formed a tight-knit community, working together to ensure their survival and happiness.

Every day, the mice would venture out of their mouse hole in search of food and adventure. They would scurry through fields, climb trees, and explore every nook and cranny of their village. Despite their small size, the mice were determined and fearless, always ready to overcome any obstacle that came their way.

On

### Async Streaming Chat

In [13]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

=== Calling Function ===
Calling function: add with args: {
  "a": 121,
  "b": 8
}
Got output: 129
The answer to 121 plus 8 is 129.

Once upon a time, in a lush green meadow, there was a group of 129 mice who lived in perfect harmony. These mice were known for their adventurous spirit and their love for exploration.

The leader of the mouse group was a wise and charismatic mouse named Oliver. Oliver was respected by all the mice for his knowledge and ability to guide the group through any challenge they encountered. Under his leadership, the mice formed a close-knit community, always looking out for one another.

Every day, the mice would embark on exciting expeditions, venturing into the unknown corners of the meadow. They would scurry through tall grasses, climb trees, and navigate through intricate tunnels, always eager to discover new places and treasures.

One sunny morning, as the mice were exploring a dense thicket, they stumbled upon a hidden treasure chest. The chest was fille

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)